In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
import seaborn as sns 
import numpy as np

In [ ]:
crimes = pd.read_csv("data/Crime_Data_2010_2017_clean2.csv")

In [ ]:
crimesTimeName = []

for i in crimes['Time Occurred']:
    if i in range(500, 1159):
        crimesTimeName.append('Morning')
    elif i in range(1200,1659):
        crimesTimeName.append('Afternoon')
    elif i in range(1700, 2059):
        crimesTimeName.append('Evening')
    else:
        crimesTimeName.append('Night')

crimes['Time of Day'] = crimesTimeName
del crimesTimeName

In [ ]:
display(crimes.head())

In [ ]:
crimes.loc[crimes['Victim Sex'].isna(), 'Victim Sex'] = 2
crimes.loc[:, 'Victim Sex'] = crimes.loc[:, 'Victim Sex'].astype(int)

crimes.loc[crimes['Victim Descent'].isna(), 'Victim Descent'] = 0
crimes.loc[:, 'Victim Descent'] = crimes.loc[:, 'Victim Descent'].astype(int)

In [ ]:
#crimes.fillna(crimes.mean(), inplace=True)
crimes.dropna()

In [ ]:
print(crimes.isna().sum())

In [ ]:
crimes.columns

In [ ]:
len(crimes)

In [ ]:
crimes.sample(n=1000, replace=True)

In [ ]:
X = crimes[['Area ID',
       'Reporting District', 'Crime Code', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code',
       'Weapon Used Code', 'Longitude', 'Latitude']]

#X = crimes.drop(columns = ['Time of Day'])
#X = crimes.drop(columns = ['Time of Day', 'Month', 'Day', 'Year', 'Time Occurred', 'Longitude', 'Latitude'])

y = crimes[['Time of Day']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
#Import knearest neighbors Classifier model
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier with k = 5 neighbors
knn = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
knn.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = knn.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Import confusion matrix and classification report
display(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))

'''for k in range(0,30):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print(k,"Accuracy:",metrics.accuracy_score(y_test, y_pred))'''

# The GridsearchCV model with KNN came out to 41.3% but the results were lost due to me trying to run it again. I have not been able to get it running again because it takes over 4 hours and jupyter will freeze occasionally.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

#Ref: https://medium.com/@erikgreenj/k-neighbors-classifier-with-gridsearchcv-basics-3c445ddeb657

ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

model = KNeighborsClassifier()
#model = RandomForestClassifier()

params = {
    'n_neighbors':[3, 5, 11, 19],
    'weights':['uniform', 'distance'],
    'metric':['euclidean', 'manhattan']
}

# Params from: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
'''params = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}'''

gs = GridSearchCV(
    model,
    params,
    verbose=2,
    cv=3,
    n_jobs=-1
)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#gs = RandomizedSearchCV(estimator = model, param_distributions = params, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs=-1)

gs_results = gs.fit(X_train, y_train)

In [ ]:
gs_results.best_score_

In [ ]:
gs_results.best_estimator_

In [ ]:
gs_results.best_params_